In [ ]:
!pip install transformers faster_whisper torchaudio torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.

In [ ]:
!unzip wavs2.zip

In [ ]:
import os
import subprocess
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from faster_whisper import WhisperModel
import torchaudio
import torch

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

2.3.0+cu121
True
Using device: cuda


In [ ]:
model_id = "VladS159/Whisper_medium_ro_VladS_10000_steps_multi_gpu_07_03_2024"
output_file_path = "./transcriptions.txt"

In [ ]:
!ct2-transformers-converter --model {model_id} --output_dir model_vlad --copy_files tokenizer_config.json preprocessor_config.json --quantization float16
model = WhisperModel("model_vlad", device=str(device), compute_type="float16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
config.json: 100% 1.32k/1.32k [00:00<00:00, 7.56MB/s]
model.safetensors: 100% 1.63G/1.63G [00:13<00:00, 120MB/s]
generation_config.json: 100% 3.77k/3.77k [00:00<00:00, 21.7MB/s]
tokenizer_config.json: 100% 283k/283k [00:00<00:00, 1.49MB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 3.43MB/s]
merges.txt: 100% 494k/494k [00:00<00:00, 7.67MB/s]
normalizer.json: 100% 52.7k/52.7k [00:00<00:00, 870kB/s]
added_tokens.json: 100% 34.6k/34.6k [00:00<00:00, 127MB/s]
special_tokens_map.json: 100% 2.19k/2.19k [00:00<00:00, 17.2MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associat

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

In [ ]:
def transcribe_audio(audio_file_path, file_index):
  print(audio_file_path)
  segments, info = model.transcribe(audio_file_path, beam_size=5, language="ro", condition_on_previous_text=False)

  transcription = ""
  for segment in segments:
      transcription += segment.text

  with open(output_file_path, 'a', encoding='utf-8') as f:
    print(transcription)
    final_transcription =  f"wavs/{file_index+142}.wav|{transcription}"
    f.write(final_transcription + '\n')

In [ ]:
audio_folder_path = "./wavs2"
audio_files = [f for f in os.listdir(audio_folder_path) if f.endswith('.wav')]
starting_index = 142

for i, _ in enumerate(audio_files):
    audio_path = f"./wavs2/{i + starting_index}.wav"
    transcribe_audio(audio_path, i)

./wavs2/142.wav
O masă de aer tropical se extinde și se va intensifica din nou în zona țării noastre.
./wavs2/143.wav
 vom avea temperaturi din ce în ce mai ridicate.
./wavs2/144.wav
Încă de azi vom vorbi despre temperaturi mai ridicate decât ieri.
./wavs2/145.wav
Mâine, ceva mai cald de atât, adică maxim treizeci sau treizeci și trei de grade.
./wavs2/146.wav
Azi ne așteptăm la o maximă în București, de douăzeci și opt sau douăzeci și nouă de grade.
./wavs2/147.wav
 mâine, ne așteptăm la o temperatură chiar mai ridicată de atât.
./wavs2/148.wav
Vremea va deveni mai călduroasă, temperatura maximă se va ridica.
./wavs2/149.wav
Vom avea temperaturii ridicate și disconfort termic din ce în ce mai accentuat.
./wavs2/150.wav
Un borbat pasionat de istorie din China a făcut o descoperire alarmantă.
./wavs2/151.wav
 După ce a plătit mai puțin de un dolar pentru patru cărți pe care le-a găsit.
./wavs2/152.wav
A găsit patru cărţi la un centrul de reciclare, iar cărţile conţineau documente cu inf

In [ ]:
print(f"Transcriptions saved to {output_file_path}")

Transcriptions saved to ./transcriptions.txt
